# 异步计算

In [15]:
import time
class Benchmark(): # 本类已保存在 gluonbook 包中⽅便以后使⽤。
    def __init__(self, prefix=None):
        self.prefix = prefix + ' ' if prefix else ''
    def __enter__(self):
        self.start = time.time()
    def __exit__(self, *args):
        print('%stime: %.4f sec' % (self.prefix, time.time() - self.start))

In [16]:
import mxnet as mx 
from mxnet import nd

In [17]:
with Benchmark('Workloads are queued.'):
    x = nd.random.uniform(shape=(2000, 2000))
    y = nd.dot(x, x).sum()
with Benchmark('Workloads are finished.'):
    print('sum =', y)

Workloads are queued. time: 0.0040 sec
sum = 
[1.9992904e+09]
<NDArray 1 @cpu(0)>
Workloads are finished. time: 0.4668 sec


In [18]:
with Benchmark():
    y = nd.dot(x, x)
    y.wait_to_read()

time: 0.2623 sec


In [19]:
with Benchmark():
    y = nd.dot(x, x)
    z = nd.dot(x, x)
    nd.waitall()

time: 0.5914 sec


In [20]:
with Benchmark():
    y = nd.dot(x, x)
    y.asnumpy()

time: 0.2563 sec


In [21]:
with Benchmark():
    y = nd.dot(x, x)
    y.norm().asscalar()

time: 0.3491 sec


In [22]:
with Benchmark('synchronous.'):
    for _ in range(1000):
        y = x + 1
        y.wait_to_read()
        
with Benchmark('asynchronous.'):
    for _ in range(1000):
        y = x + 1
    nd.waitall()

synchronous. time: 6.8108 sec
asynchronous. time: 7.1534 sec


MXNet 包括⽤⼾直接⽤来交互的前端和系统⽤来执⾏计算的后端。
+ MXNet 能够通过异步计算提升计算性能。
+ 我们建议使⽤每个小批量训练或预测时⾄少使⽤⼀个同步函数，从而避免在短时间内将过多计算任务丢给后端。